In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [3]:
!pip install -q dagshub mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 52.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 20.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [ ]:
!pip install --upgrade scikit-learn==1.3.2 imbalanced-learn==0.11.0

In [4]:
import dagshub
dagshub.init(repo_owner='azhgh22', repo_name='ML2_fraud_detection', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=666e2257-d3cd-4ca0-a210-cc0e86399987&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6b491e42ad36504d1288f6f6e97bd1b16b0fb7c290f705ad748515eaa2dae2c3




Output()

Accessing as azhgh22

Initialized MLflow to track repo "azhgh22/ML2_fraud_detection"

Repository azhgh22/ML2_fraud_detection initialized!

In [5]:
trans_data = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

In [7]:
identity_data = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')

In [8]:
new_cols = []
for col in identity_data.columns:
    new_col = col.replace('-', '_')
    new_cols.append(new_col)
identity_data.columns = new_cols

In [9]:
data = pd.merge(trans_data,identity_data,on='TransactionID',how='left')

In [10]:
import mlflow
import sklearn
logged_model = 'runs:/5a666e327b1d418bb8d027bde528f88d/xgboost_with_identity'

# Load model as a PyFuncModel.
loaded_model = mlflow.sklearn.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
predicted = loaded_model.predict_proba(pd.DataFrame(data))

selector dropped 1
Na dropped 12 features
var dropped :  239
corr dropped:  27


In [11]:
predicted

array([[0.99586143, 0.00413857],
       [0.99235743, 0.00764257],
       [0.98948212, 0.01051788],
       ...,
       [0.99611774, 0.00388226],
       [0.99349098, 0.00650902],
       [0.99245773, 0.00754227]])

In [12]:
submission = pd.DataFrame({'TransactionID': data['TransactionID'], 'isFraud': predicted[:,1]})  # Replace 'test_ids' and 'predictions'

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)